[View in Colaboratory](https://colab.research.google.com/github/DroidRonin/Pima-Indian-Diabetes-Classification/blob/master/Seq2Seq.ipynb)

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.


In [0]:
from google.colab import files


In [0]:
uploaded = files.upload()


Saving fra1.txt to fra1.txt


In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

User uploaded file "fra1.txt" with length 9746530 bytes


In [0]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(fn, 'r', encoding='utf-8') as f:
  
    lines = f.read().split('\n')
    

for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


In [0]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 73
Max sequence length for inputs: 16
Max sequence length for outputs: 18


In [0]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [0]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 12s 1ms/step - loss: 2.1439 - val_loss: 2.5532
Epoch 2/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.7374 - val_loss: 1.9232
Epoch 3/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.4532 - val_loss: 1.7011
Epoch 4/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.2977 - val_loss: 1.6804
Epoch 5/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.1811 - val_loss: 1.5485
Epoch 6/100
8000/8000 [==============================] - 11s 1ms/step - loss: 1.0817 - val_loss: 1.3834
Epoch 7/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.9869 - val_loss: 1.4228
Epoch 8/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.8877 - val_loss: 1.3477
Epoch 9/100
8000/8000 [==============================] - 11s 1ms/step - loss: 0.7991 - val_loss: 1.2638
Epoch 10/100
800

/usr/local/lib/python3.6/dist-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [0]:
  encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [0]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence



In [0]:
for seq_index in range(5000,5200):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)
    

-
Input sentence: I finally won.
Decoded sentence: I finally won.

-
Input sentence: I finished it.
Decoded sentence: I finished it.

-
Input sentence: I forbid that.
Decoded sentence: I forbid that.

-
Input sentence: I forgive you.
Decoded sentence: I forgive you.

-
Input sentence: I fought back.
Decoded sentence: I fought back.

-
Input sentence: I found job.
Decoded sentence: I found a job.

-
Input sentence: I found these.
Decoded sentence: I found these.

-
Input sentence: I found these.
Decoded sentence: I found these.

-
Input sentence: I freaked out.
Decoded sentence: I freaked out.

-
Input sentence: I freaked out.
Decoded sentence: I freaked out.

-
Input sentence: I freaked out.
Decoded sentence: I freaked out.

-
Input sentence: I freaked out.
Decoded sentence: I freaked out.

-
Input sentence: I got bonus.
Decoded sentence: I got a bonus.

-
Input sentence: I got it free.
Decoded sentence: I got it free.

-
Input sentence: I got married.
Decoded sentence: I got married.


In [0]:
import pandas as pd

In [0]:
output = pd.DataFrame(decoded_sentence)

ValueError: ignored

In [0]:
prediction = pd.DataFrame(decoded_sentence, columns=['predictions']).to_csv('prediction.csv')

ValueError: ignored

In [0]:
LI = []


In [0]:
LI.append(decoded_sentence)

In [0]:
prediction = pd.DataFrame(LI, columns=['predictions']).to_csv('prediction.csv')

In [0]:
from google.colab import files
files.download('prediction.csv')

['Drop it!\n']

Drop it!



TypeError: ignored

In [0]:
with open("fra1.txt") as f:
  num = f.readlines()
  print(len(num))
  

154883


In [0]:
mylist = list()

In [0]:
input_list = list()

In [0]:
for seq_index in range(len(num)):
  decoded_sentence = decode_sequence(input_seq)
  mylist.append(decoded_sentence)